**1. AUTO FILL DATASET DIPLOY (GENERATE DATA SINTETIS UNTUK KHUSUS KOLOM KOSONG SAJA PADA DATASET DIPLOY DENGAN LLM GPT5.1)**

In [42]:
import os
import json
import pandas as pd
import asyncio
import nest_asyncio
from typing import List, Dict, Any
from tqdm.asyncio import tqdm_asyncio
from tenacity import retry, wait_exponential, stop_after_attempt
from datetime import datetime
from openai import OpenAI, AsyncOpenAI

In [51]:
DRIVE_DATASET_DIR = "/Users/irz/projects/dtp-data-pipeline"

# ============================================
# KONFIGURASI
# ============================================

# API OpenRouter
os.environ["OPENAI_API_KEY"] = "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"
client = OpenAI(base_url="https://openrouter.ai/api/v1")
async_client = AsyncOpenAI(base_url="https://openrouter.ai/api/v1")

# Model LLM dari OpenRouter
MODEL_LLM = "google/gemini-2.5-flash"

CONCURRENCY = 10  # Concurrent API calls (reduced for better control)
CHECKPOINT_SIZE = 500  # Save setiap 600 baris
REQUEST_TIMEOUT = 600  # 10 minutes timeout

# Global semaphore for concurrency control
GLOBAL_SEMAPHORE = None  # Will be initialized after asyncio event loop is ready

# File paths
INPUT_FILE  = f"{DRIVE_DATASET_DIR}/Pipeline Flagging/Data Diploy Flagged/Data_Diploy_Cleaned.xlsx"
OUTPUT_FILE = f"{DRIVE_DATASET_DIR}/modified_dataloker.xlsx"
REF_FILE    = f"{DRIVE_DATASET_DIR}/data_need_to_generate.xlsx"

In [52]:
# df = pd.read_excel(INPUT_FILE)
# df = df[df['Area_Fungsi'] == "Sains Data-Kecerdasan Artifisial"].reset_index(drop=True)
# # df.Area_Fungsi.unique()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4440 entries, 0 to 4439
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Jenjang_Pendidikan                  4440 non-null   object
 1   Jurusan                             4440 non-null   object
 2   Judul_Tugas_Akhir                   4440 non-null   object
 3   Bidang_Pelatihan                    4440 non-null   object
 4   Nama_Pelatihan                      4440 non-null   object
 5   Sertifikasi                         4440 non-null   object
 6   Bidang_Sertifikasi                  4440 non-null   object
 7   Posisi_Pekerjaan                    4440 non-null   object
 8   Deskripsi_tugas_dan_tanggung_jawab  4440 non-null   object
 9   Lama_Bekerja                        4440 non-null   object
 10  Keterampilan                        4440 non-null   object
 11  Area_Fungsi                         4440 non-null   obje

In [53]:
import random

def pilih_level(row, requirements_df):
    """
    Memilih level secara random dari requirements berdasarkan area fungsi dan slot yang tersedia.
    
    Args:
        row: Baris data dari input file
        requirements_df: DataFrame requirements yang berisi Area_Fungsi, Level, dan Slot
    
    Returns:
        Level yang dipilih (int atau str), atau None jika tidak ada slot tersedia
    """
    area_fungsi = row.get("Area_Fungsi", "")
    
    # Normalisasi: lowercase untuk case-insensitive matching
    area_fungsi_lower = area_fungsi.lower().strip()
    
    # Filter requirements berdasarkan area fungsi yang sama (case-insensitive)
    matching_reqs = requirements_df[
        (requirements_df["Area_Fungsi"].str.lower().str.strip() == area_fungsi_lower) & 
        (requirements_df["sisa_slot"] > 0)
    ]
    
    if matching_reqs.empty:
        return None
    
    # Pilih random dari level yang tersedia
    selected_row = matching_reqs.sample(n=1).iloc[0]
    return selected_row["Level_Okupasi"]

In [54]:
def update_slot(area_fungsi, level, requirements_df):
    """
    Mengurangi slot sebanyak 1 untuk area fungsi dan level tertentu.
    
    Args:
        area_fungsi: Area fungsi yang dipilih
        level: Level yang dipilih
        requirements_df: DataFrame requirements (akan dimodifikasi in-place)
    
    Returns:
        True jika berhasil update, False jika tidak ditemukan
    """
    # Normalisasi: lowercase untuk case-insensitive matching
    area_fungsi_lower = area_fungsi.lower().strip()
    
    mask = (
        (requirements_df["Area_Fungsi"].str.lower().str.strip() == area_fungsi_lower) & 
        (requirements_df["Level_Okupasi"] == level)
    )
    
    if mask.any():
        idx = requirements_df[mask].index[0]
        current_slot = requirements_df.at[idx, "sisa_slot"]
        
        if current_slot > 0:
            requirements_df.at[idx, "sisa_slot"] = current_slot - 1
            return True
    
    return False

In [55]:
# ============================================
# SYSTEM PROMPT & USER PROMPT (REVERSE FLAGGING)
# ============================================

system_prompt = """
Anda adalah AI Expert dalam bidang Teknologi Informasi dan Komunikasi (TIK) yang ahli dalam Peta Okupasi Nasional TIK (PON TIK 2025).
Tugas Anda adalah MEMPERBAIKI data talenta agar KONSISTEN dan REALISTIS dengan Area Fungsi dan Level Okupasi yang DIBERIKAN.
Anda tidak melakukan klasifikasi area fungsi/level, tetapi MENYESUAIKAN PROFIL agar cocok dengan area fungsi dan level target.
Output Anda harus SELALU berupa JSON yang valid (tanpa teks lain di luar JSON).
"""

def make_prompt(row, expected_level):
    """
    Membuat prompt untuk reverse flagging dengan current level dan expected level.
    
    Args:
        row: Baris data dari dataframe
        expected_level: Level yang diinginkan dari requirements
    
    Returns:
        String prompt untuk LLM
    """
    area_fungsi = row.get("Area_Fungsi", "")
    current_level = row.get("Level_Okupasi", "")

    existing_data = {
        "Jenjang_Pendidikan": row.get("Jenjang_Pendidikan", ""),
        "Jurusan": row.get("Jurusan", ""),
        "Judul_Tugas_Akhir": row.get("Judul_Tugas_Akhir", ""),
        "Bidang_Pelatihan": row.get("Bidang_Pelatihan", ""),
        "Nama_Pelatihan": row.get("Nama_Pelatihan", ""),
        "Sertifikasi": row.get("Sertifikasi", ""),
        "Bidang_Sertifikasi": row.get("Bidang_Sertifikasi", ""),
        "Posisi_Pekerjaan": row.get("Posisi_Pekerjaan", ""),
        "Deskripsi_tugas_dan_tanggung_jawab": row.get("Deskripsi_tugas_dan_tanggung_jawab", ""),
        "Lama_Bekerja": row.get("Lama_Bekerja", ""),
        "Keterampilan": row.get("Keterampilan", "")
    }

    return f"""
TARGET PROFIL YANG DIINGINKAN (HASIL FLAGGING):
Area Fungsi TIK: {area_fungsi}
Level Okupasi Saat Ini: {current_level}
Level Okupasi yang Diinginkan: {expected_level}

DATA TALENTA SAAT INI (SUDAH BENAR UNTUK LEVEL INI, PERLU DIUBAH KE LEVEL YANG DIINGINKAN):
Jenjang Pendidikan: {existing_data['Jenjang_Pendidikan']}
Jurusan: {existing_data['Jurusan']}
Judul Tugas Akhir: {existing_data['Judul_Tugas_Akhir']}
Bidang Pelatihan: {existing_data['Bidang_Pelatihan']}
Nama Pelatihan: {existing_data['Nama_Pelatihan']}
Sertifikasi: {existing_data['Sertifikasi']}
Bidang Sertifikasi: {existing_data['Bidang_Sertifikasi']}
Posisi Pekerjaan: {existing_data['Posisi_Pekerjaan']}
Deskripsi Tugas dan Tanggung Jawab: {existing_data['Deskripsi_tugas_dan_tanggung_jawab']}
Lama Bekerja: {existing_data['Lama_Bekerja']}
Keterampilan: {existing_data['Keterampilan']}

TUGAS ANDA (REVERSE FLAGGING):
1. Area Fungsi dan Level Okupasi yang Diinginkan di atas adalah TARGET yang HARUS DIPENUHI.
2. Periksa SEMUA kolom data talenta.
3. Jika suatu kolom SUDAH VALID dan SUDAH SELARAS dengan Area Fungsi:
   → MODIFIKASI SESUAI LEVEL YANG DIINGINKAN.
4. Jika kolom KOSONG, TIDAK VALID, TERLALU RENDAH/TERLALU TINGGI, atau TIDAK RELEVAN:
   → PERBAIKI / GANTI agar PROFIL TALENTA MENJADI:
     - Konsisten dengan Area Fungsi TIK target
     - Konsisten dengan Level Okupasi yang Diinginkan (1–9)
     - Realistis, memungkinkan ada kualifikasi talenta seperti itu di dunia nyata
     - Kualifikasi lebih rendah atau lebih tinggi dari saat ini sesuai dari level saat ini ke level yang diinginkan menaik atau menurun
     - Jika level yang diinginkan lebih tinggi dari level saat ini → kualifikasi harus MENINGKAT (tambahkan sertifikasi, pelatihan, pengalaman pekerjaan, tingkat pendidikan)
     - Jika level yang diinginkan lebih rendah dari level saat ini → kualifikasi harus MENURUN (kurangi atau hilangkan sertifikasi, pelatihan, pengalaman pekerjaan, tingkat pendidikan)
5. Anda HARUS menyesuaikan profil agar cocok dengan Level yang Diinginkan ({expected_level}), bukan Level Saat Ini.

PANDUAN AREA FUNGSI TIK (PON TIK 2025):

1. Tata Kelola Teknologi Informasi (Level 2–9)
   - Fokus: IT Governance, IT Strategy, Enterprise Architecture, IT Risk Management, IT Compliance
   - Contoh Posisi: CIO, IT Manager, Enterprise Architect, IT Auditor, IT Governance Specialist
   - Keterampilan: COBIT, ITIL, ISO 27001, Risk Management, Strategic Planning, Compliance

2. Pengembangan Produk Digital (Level 2–9)
   - Fokus: Software Development, Web/Mobile App, UI/UX Design, Product Management
   - Contoh Posisi: Software Developer, Full Stack Developer, Frontend/Backend Developer, Mobile Developer, Product Manager
   - Keterampilan: Programming (Java, Python, JavaScript, dll), Framework (Laravel, React, Vue, Flutter), Database, Git, Agile

3. Sains Data-Kecerdasan Artifisial (Level 2–9)
   - Fokus: Data Science, Machine Learning, AI, Big Data, Data Analytics, Data Engineering
   - Contoh Posisi: Data Scientist, ML Engineer, Data Analyst, AI Researcher, Data Engineer
   - Keterampilan: Python, R, TensorFlow, PyTorch, SQL, Statistics, Machine Learning, Deep Learning, Data Visualization

4. Keamanan Informasi Dan Siber (Level 3–9)
   - Fokus: Cybersecurity, Information Security, Ethical Hacking, Security Audit, SOC
   - Contoh Posisi: Security Analyst, Ethical Hacker, SOC Analyst, Security Consultant, CISO
   - Keterampilan: Network Security, Penetration Testing, SIEM, Firewall, IDS/IPS, ISO 27001, Incident Response

5. Teknologi Dan Infrastruktur (Level 4–9)
   - Fokus: Network Infrastructure, Cloud Computing, Server Management, DevOps, System Administration
   - Contoh Posisi: Network Engineer, System Administrator, Cloud Architect, DevOps Engineer, Infrastructure Manager
   - Keterampilan: Networking (Cisco, Juniper), Cloud (AWS, Azure, GCP), Linux/Windows Server, Virtualization, Docker, Kubernetes

6. Layanan Teknologi Informasi (Level 1–8)
   - Fokus: IT Support, Help Desk, Technical Support, IT Service Management
   - Contoh Posisi: IT Support, Help Desk, Technical Support Engineer, IT Service Manager
   - Keterampilan: Troubleshooting, Customer Service, ITIL, Ticketing System, Hardware/Software Support

PANDUAN LEVEL OKUPASI (UNTUK MENENTUKAN "KEDALAMAN" PROFIL):

Level 1–3 (Entry Level / Junior):
- Pendidikan: SD/SMP/SMA/SMK/D3 (fresh graduate atau pengalaman <1 tahun)
- Pengalaman: 0–1 tahun
- Contoh: Junior IT Support, Junior Developer, Magang
- Keterampilan: 6–8 skill dasar
- Sertifikasi: Boleh belum punya atau basic

Level 4–5 (Junior–Intermediate):
- Pendidikan: D3/D4/S1
- Pengalaman: 1–3 tahun
- Keterampilan: 8–10 skill
- Sertifikasi: 1–2 sertifikasi profesional

Level 5–6 (Intermediate / Staff):
- Pendidikan: S1 (pengalaman 3–6 tahun), S2 (pengalaman 1–2 tahun)
- Pengalaman: 3–6 tahun
- Keterampilan: 10–12 skill
- Sertifikasi: 2–3 sertifikasi profesional

Level 7–8 (Senior / Specialist / Supervisor):
- Pendidikan: S1 (pengalaman 6–10 tahun), S2 (pengalaman 3+ tahun)
- Pengalaman: 6–10 tahun
- Keterampilan: 12–15 skill, termasuk leadership/strategic
- Sertifikasi: 3–4 sertifikasi advanced/expert

Level 9 (Expert / C-Level / Director):
- Pendidikan: S2 (pengalaman 10+ tahun), S3 (pengalaman 10+ tahun)
- Pengalaman: 10+ tahun
- Keterampilan: ≥15 skill, sangat strategis
- Sertifikasi: 4+ sertifikasi expert/international

ATURAN PER KOLOM (KHUSUS UNTUK REVERSE FLAGGING):

1️⃣ Jenjang_Pendidikan:
   - Sesuaikan dengan Level Okupasi yang Diinginkan (lihat panduan level).
   - Jika sudah logis untuk level tersebut → pertahankan.
   - Jika terlalu rendah/tinggi → sesuaikan.

2️⃣ Jurusan:
   - WAJIB relevan dengan Area Fungsi.
   - Gunakan nama lengkap (bukan singkatan).

3️⃣ Judul_Tugas_Akhir:
   - Untuk selain pendidikan tinggi (SD/SMP/SMA/SMK): "Tidak ada tugas akhir".
   - Untuk mahasiswa (D3/D4/S1/S2/S3): "Belum lulus"
   - Untuk lulusan D3/D4/S1/S2/S3: buat judul spesifik, realistis, natural, dan relevan dengan Area Fungsi.
   - Fokus D3/D4 (Vokasi): Arahkan judul pada Applied AI dan Data Engineering. Tekankan implementasi sistem, deployment ke perangkat (IoT/Mobile), atau dashboard cerdas untuk industri.
   - Fokus S1 (Sarjana): Arahkan pada Analytical Problem Solving. Judul wajib memuat komparasi model yang kritis, feature engineering mendalam, atau analisis performa pada kasus nyata.
   - Fokus S2/S3 (Pascasarjana): Wajib mengandung unsur Novelty. Judul harus mengeksplorasi modifikasi arsitektur, Hybrid-Model, Explainable AI (XAI), atau teori baru pada data kompleks.
   - Larangan Format Klise: DILARANG menggunakan format pasif "Penerapan Algoritma X pada Y". Gunakan kata kerja solusi: "Optimasi...", "Deteksi Dini Berbasis...", atau "Personalisasi Adaptif...".
   - Integrasi Tren Teknologi: Dorong penggunaan teknologi terkini jika relevan, seperti Large Language Models (LLM), Generative AI, Computer Vision, atau Federated Learning.
   - Spesifisitas Data: Judul harus menyiratkan tantangan pada jenis datanya (misal: data sensor time-series, teks hukum tidak terstruktur, data genomik, atau citra satelit).
   - Kontekstualisasi Domain: Hindari judul generik. Kaitkan metode AI secara eksplisit dengan domain unik (misal: Precision Agriculture, Cyber-Security, Fintech, atau Smart Health).

4️⃣ Bidang_Pelatihan & Nama_Pelatihan:
   - Relevan dengan Area Fungsi.
   - Semakin tinggi level, semakin advanced.

5️⃣ Sertifikasi & Bidang_Sertifikasi:
   - Jumlah & level sertifikasi mengikuti Level Okupasi yang Diinginkan. Mungkin tidak punya sertifikasi di level rendah.
   - Relevan dengan Area Fungsi.
   - Jika profil realistis tanpa sertifikasi (misal level rendah) → gunakan frasa seperti "Belum memiliki sertifikasi" tetapi tetap konsisten.

6️⃣ Posisi_Pekerjaan:
   - Kualifikasi untuk level yang rendah memungkinkan untuk diisi 'Belum memiliki pengalaman kerja'
   - Jika ada, harus selaras dengan Area Fungsi dan Level yang Diinginkan.
   - Tidak boleh singkatan.

7️⃣ Deskripsi_tugas_dan_tanggung_jawab:
   - 2–3 kalimat yang mencerminkan Area Fungsi dan Level yang Diinginkan, atau 'Belum memiliki pengalaman kerja'.

8️⃣ Lama_Bekerja:
   - Format: "X tahun Y bulan Z hari".
   - Durasi disesuaikan dengan Level yang Diinginkan (lihat panduan level).

9️⃣ Keterampilan:
   - 5–15+ skill tergantung level yang diinginkan.
   - Dominan relevan dengan Area Fungsi.
   - Kombinasi hard skill, tools/tech, soft skill.
   - Tanpa singkatan yang ambigu.

CONTOH (REVERSE FLAGGING):
TARGET: Area Fungsi = "Sains Data-Kecerdasan Artifisial", Level yang Diinginkan = 6
→ Hasil akhir harus tampak seperti profil seorang Data Scientist level menengah (pendidikan, pengalaman, pelatihan, sertifikasi, posisi, deskripsi, keterampilan semuanya masuk akal).

OUTPUT FORMAT (WAJIB JSON VALID, TANPA TEKS LAIN):

{{
  "Jenjang_Pendidikan": "[Sesuai level yang diinginkan, pilih satu antara SD/SMP/SMA/SMK/D3/D4/S1/S2/S2/S3]",
  "Jurusan": "[Relevan dengan area fungsi]",
  "Judul_Tugas_Akhir": "[Relevan atau 'Tidak ada tugas akhir']",
  "Bidang_Pelatihan": "[Sesuai area fungsi atau 'Tidak ada pelatihan']",
  "Nama_Pelatihan": "[Spesifik dan relevan atau 'Tidak ada pelatihan']",
  "Sertifikasi": "[Sesuai area fungsi dan level yang diinginkan atau 'Belum memiliki sertifikasi']",
  "Bidang_Sertifikasi": "[Sesuai sertifikasi atau 'Tidak ada sertifikasi']",
  "Posisi_Pekerjaan": "[Sesuai area fungsi dan level yang diinginkan, tanpa singkatan. atau 'Belum memiliki pengalaman kerja']",
  "Deskripsi_tugas_dan_tanggung_jawab": "[2–3 kalimat sesuai posisi dan level yang diinginkan, atau 'Belum memiliki pengalaman kerja']",
  "Lama_Bekerja": "[Format: X tahun Y bulan Z hari, sesuai level yang diinginkan, atau '0 tahun 0 bulan 0 hari' jika Belum memiliki pengalaman kerja]",
  "Keterampilan": "[Minimal 5–15 skill, relevan dengan area fungsi, tanpa singkatan]"
}}

⚠️ CRITICAL RULES:
1. Area Fungsi dan Level yang Diinginkan adalah TARGET yang HARUS DIPENUHI.
2. Jika data existing sudah sesuai dengan target → PERTAHANKAN.
3. Jika data existing TIDAK SESUAI atau KOSONG → SESUAIKAN agar match dengan Level yang Diinginkan ({expected_level}).
4. Output HANYA JSON (tanpa kalimat penjelasan).
5. TIDAK BOLEH ada placeholder invalid: "Unknown", "-", "N/A", dll.
6. Data harus REALISTIS dan konsisten dengan dunia kerja.
7. Pendidikan hanya pilih salah satu dari: SD, SMP, SMA, SMK, D3, D4, S1, S2, S3. Tidak boleh ada ambiguitas, misal SMA/SMK

Sekarang sesuaikan profil talenta di atas agar MATCH dengan Area Fungsi "{area_fungsi}" dan Level yang Diinginkan "{expected_level}" dalam format JSON seperti spesifikasi:
"""

In [56]:
# ============================================
# WORKER & BATCH PROCESSING
# ============================================

@retry(wait=wait_exponential(multiplier=1, min=2, max=30), stop=stop_after_attempt(5))
async def call_openrouter(prompt, row_index):
    """
    Memanggil OpenRouter API secara async dengan timeout handling
    """
    try:
        # Use asyncio.wait_for for timeout handling
        response = await asyncio.wait_for(
            async_client.chat.completions.create(
                model=MODEL_LLM,
                temperature=0,
                response_format={"type": "json_object"},
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": prompt}
                ]
            ),
            timeout=REQUEST_TIMEOUT
        )
        
        return response.choices[0].message.content.strip()
    
    except asyncio.TimeoutError:
        print(f"[TIMEOUT] Row {row_index}: Request exceeded {REQUEST_TIMEOUT}s", flush=True)
        raise
    except Exception as e:
        print(f"[ERROR] Row {row_index}: {e}", flush=True)
        raise

def validate_json(s):
    try:
        return json.loads(s)
    except:
        return None

def safe_get(data, key, default=""):
    """Safely get value and convert to string"""
    val = data.get(key, default)
    if isinstance(val, (list, dict)):
        return str(val)  # Convert complex types to string
    return str(val) if val is not None else default

async def worker(row_index, row, df, requirements_df, max_retries=3):
    """Process single row with global semaphore control and retry logic"""
    global GLOBAL_SEMAPHORE
    if GLOBAL_SEMAPHORE is None:
        GLOBAL_SEMAPHORE = asyncio.Semaphore(CONCURRENCY)
    
    async with GLOBAL_SEMAPHORE:
        for attempt in range(1, max_retries + 1):
            try:
                # Pilih level dari requirements
                expected_level = pilih_level(row, requirements_df)
                
                if expected_level is None:
                    print(f"[SKIP] Row {row_index}: Tidak ada slot tersedia untuk Area Fungsi '{row.get('Area_Fungsi', '')}'", flush=True)
                    return False
                
                # Buat prompt dengan expected level
                prompt = make_prompt(row, expected_level)
                text = await call_openrouter(prompt, row_index)

                if not text:
                    raise ValueError("Response kosong dari API")

                data = validate_json(text)

                if not data:
                    clean = text.replace("```json","").replace("```","").strip()
                    data = validate_json(clean)

                if not data:
                    raise ValueError("Gagal parse JSON dari response")

                # UPDATE KOLOM TALENTA dengan safe_get untuk menghindari error type
                df.at[row_index, "Jenjang_Pendidikan"] = safe_get(data, "Jenjang_Pendidikan")
                df.at[row_index, "Jurusan"] = safe_get(data, "Jurusan")
                df.at[row_index, "Judul_Tugas_Akhir"] = safe_get(data, "Judul_Tugas_Akhir")
                df.at[row_index, "Bidang_Pelatihan"] = safe_get(data, "Bidang_Pelatihan")
                df.at[row_index, "Nama_Pelatihan"] = safe_get(data, "Nama_Pelatihan")
                df.at[row_index, "Sertifikasi"] = safe_get(data, "Sertifikasi")
                df.at[row_index, "Bidang_Sertifikasi"] = safe_get(data, "Bidang_Sertifikasi")
                df.at[row_index, "Posisi_Pekerjaan"] = safe_get(data, "Posisi_Pekerjaan")
                df.at[row_index, "Deskripsi_tugas_dan_tanggung_jawab"] = safe_get(data, "Deskripsi_tugas_dan_tanggung_jawab")
                df.at[row_index, "Lama_Bekerja"] = safe_get(data, "Lama_Bekerja")
                df.at[row_index, "Keterampilan"] = safe_get(data, "Keterampilan")
                
                # UPDATE Level_Okupasi dengan expected_level
                df.at[row_index, "Level_Okupasi"] = expected_level
                
                # Kurangi slot di requirements
                area_fungsi = row.get("Area_Fungsi", "")
                update_slot(area_fungsi, expected_level, requirements_df)
                
                # Jika berhasil, return True
                if attempt > 1:
                    print(f"[SUCCESS] Row {row_index}: Berhasil pada percobaan ke-{attempt}", flush=True)
                return True
                
            except Exception as e:
                if attempt < max_retries:
                    print(f"[RETRY {attempt}/{max_retries}] Row {row_index}: {str(e)[:100]} - Mencoba lagi...", flush=True)
                    await asyncio.sleep(2 * attempt)  # Exponential backoff
                else:
                    print(f"[FAILED] Row {row_index}: Gagal setelah {max_retries} percobaan - {str(e)[:100]}", flush=True)
                    return False
        
        return False

async def process_batch(df, start_idx, end_idx, part_number, parts_folder, requirements_df):
    """Process satu batch data dan simpan checkpoint"""
    tasks = []
    for idx in range(start_idx, end_idx):
        if idx >= len(df):
            break
        row = df.iloc[idx]
        task = worker(idx, row, df, requirements_df)
        tasks.append(task)
    
    # Process batch dengan progress bar
    # Gunakan tqdm_asyncio untuk monitoring progress
    results = []
    for coro in tqdm_asyncio.as_completed(tasks, desc=f"Part {part_number}", total=len(tasks)):
        try:
            result = await coro
            results.append(result)
        except Exception as e:
            results.append(False)
    
    # Count success/failure
    success_count = sum(1 for r in results if r is True)
    failed_count = len(results) - success_count
    print(f"  → Success: {success_count}/{len(tasks)} rows | Failed: {failed_count} rows", flush=True)
    
    # Simpan checkpoint dengan urutan kolom yang benar
    df_part = df.iloc[start_idx:end_idx].copy()
    
    # Urutan kolom yang diinginkan
    column_order = [
        "Jenjang_Pendidikan",
        "Jurusan",
        "Judul_Tugas_Akhir",
        "Bidang_Pelatihan",
        "Nama_Pelatihan",
        "Sertifikasi",
        "Bidang_Sertifikasi",
        "Posisi_Pekerjaan",
        "Deskripsi_tugas_dan_tanggung_jawab",
        "Lama_Bekerja",
        "Keterampilan",
        "Area_Fungsi",
        "Level_Okupasi"
    ]
    
    # Reorder kolom pada checkpoint
    existing_cols = [col for col in column_order if col in df_part.columns]
    other_cols = [col for col in df_part.columns if col not in column_order]
    final_column_order = existing_cols + other_cols
    df_part = df_part[final_column_order]
    
    part_file = f"{parts_folder}/part_{part_number:03d}_rows_{start_idx+1}-{end_idx}.xlsx"
    df_part.to_excel(part_file, index=False)
    
    print(f"✅ Part {part_number} saved: {part_file}")
    return part_file


In [57]:
# ============================================
# MAIN PIPELINE
# ============================================

async def main():
    # Buat folder untuk menyimpan part files
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    parts_folder = f"{DRIVE_DATASET_DIR}/Koreksi_Parts_{timestamp}"
    os.makedirs(parts_folder, exist_ok=True)
    
    print(f"📁 Folder checkpoint: {parts_folder}")
    
    # Load data input dan requirements
    df = pd.read_excel(INPUT_FILE)
    df = df[df["Area_Fungsi"] == "Sains Data-Kecerdasan Artifisial"]
    # df = df.head(200)
    df.columns = df.columns.str.strip()
    
    requirements_df = pd.read_excel(REF_FILE)
    requirements_df.columns = requirements_df.columns.str.strip()
    
    total_rows = len(df)
    total_parts = (total_rows + CHECKPOINT_SIZE - 1) // CHECKPOINT_SIZE
    
    print(f"📊 Total data: {total_rows} baris")
    print(f"📦 Akan dibagi menjadi {total_parts} part files ({CHECKPOINT_SIZE} baris/part)")
    print(f"⚡ Concurrency: {CONCURRENCY} parallel requests")
    print(f"⏱️  Timeout: {REQUEST_TIMEOUT}s per request")
    print(f"\n📋 Requirements (Slot tersedia):")
    print(requirements_df.groupby("Area_Fungsi")["sisa_slot"].sum())
    print()

    # Kolom talenta yang akan dimodifikasi (Area_Fungsi tetap, Level_Okupasi akan diupdate)
    target_cols = [
        "Jenjang_Pendidikan",
        "Jurusan",
        "Judul_Tugas_Akhir",
        "Bidang_Pelatihan",
        "Nama_Pelatihan",
        "Sertifikasi",
        "Bidang_Sertifikasi",
        "Posisi_Pekerjaan",
        "Deskripsi_tugas_dan_tanggung_jawab",
        "Lama_Bekerja",
        "Keterampilan"
    ]

    for col in target_cols:
        if col not in df.columns:
            df[col] = ""
        df[col] = df[col].astype("string")

    # Process data dalam batch dan simpan checkpoint
    part_files = []
    
    for part_num in range(0, total_rows, CHECKPOINT_SIZE):
        start_idx = part_num
        end_idx = min(start_idx + CHECKPOINT_SIZE, total_rows)
        part_number = (part_num // CHECKPOINT_SIZE) + 1
        
        print(f"\n{'='*60}")
        print(f"🔄 Processing Part {part_number}/{total_parts}: Baris {start_idx+1} - {end_idx}")
        print(f"{'='*60}")
        
        # Process batch ini
        part_file = await process_batch(df, start_idx, end_idx, part_number, parts_folder, requirements_df)
        part_files.append(part_file)
        
        print(f"   Progress: {end_idx}/{total_rows} baris ({end_idx/total_rows*100:.1f}%)")
        
        # Tampilkan slot tersisa setiap batch
        remaining_slots = requirements_df["sisa_slot"].sum()
        print(f"   📊 Slot tersisa total: {remaining_slots}")
    
    print(f"\n{'='*60}")
    print(f"🎉 Semua part selesai diproses!")
    print(f"{'='*60}")
    print(f"📂 Part files tersimpan di: {parts_folder}")
    print(f"📋 Total part files: {len(part_files)}")
    print(f"{'='*60}\n")
    
    # Merge semua part files menjadi satu file final
    print("🔗 Menggabungkan semua part files...")
    all_parts = []
    for i, part_file in enumerate(part_files, 1):
        print(f"   Loading Part {i}/{len(part_files)}: {os.path.basename(part_file)}")
        df_part = pd.read_excel(part_file)
        all_parts.append(df_part)
    
    print(f"\n✓ Berhasil load {len(all_parts)} part files")
    
    df_final = pd.concat(all_parts, ignore_index=True)
    print(f"✓ Total baris setelah penggabungan: {len(df_final)}")
    
    # Reorder kolom sesuai urutan yang diinginkan
    column_order = [
        "Jenjang_Pendidikan",
        "Jurusan",
        "Judul_Tugas_Akhir",
        "Bidang_Pelatihan",
        "Nama_Pelatihan",
        "Sertifikasi",
        "Bidang_Sertifikasi",
        "Posisi_Pekerjaan",
        "Deskripsi_tugas_dan_tanggung_jawab",
        "Lama_Bekerja",
        "Keterampilan",
        "Area_Fungsi",
        "Level_Okupasi"
    ]
    
    # Pastikan semua kolom ada di dataframe, jika ada kolom lain tetap dipertahankan di akhir
    existing_cols = [col for col in column_order if col in df_final.columns]
    other_cols = [col for col in df_final.columns if col not in column_order]
    final_column_order = existing_cols + other_cols
    
    df_final = df_final[final_column_order]
    
    # Simpan file final
    final_file = f"{DRIVE_DATASET_DIR}/Data_Loker_Corrected_{timestamp}.xlsx"
    df_final.to_excel(final_file, index=False)
    
    # Simpan requirements yang sudah diupdate
    requirements_updated_file = f"{DRIVE_DATASET_DIR}/Requirements_Updated_{timestamp}.xlsx"
    requirements_df.to_excel(requirements_updated_file, index=False)
    
    print(f"\n{'='*60}")
    print(f"✅ SELESAI!")
    print(f"{'='*60}")
    print(f"📂 Part files: {parts_folder}")
    print(f"📄 Final file: {final_file}")
    print(f"📄 Requirements updated: {requirements_updated_file}")
    print(f"📊 Total baris: {len(df_final)}")
    print(f"\n📋 Slot tersisa per Area Fungsi:")
    print(requirements_df.groupby("Area_Fungsi")["sisa_slot"].sum())
    print(f"{'='*60}\n")
    
    return parts_folder, part_files, requirements_df

# Jalankan
nest_asyncio.apply()
parts_folder, part_files, requirements_updated = await main()


📁 Folder checkpoint: /Users/irz/projects/dtp-data-pipeline/Koreksi_Parts_20251209_102415
📊 Total data: 4440 baris
📦 Akan dibagi menjadi 9 part files (500 baris/part)
⚡ Concurrency: 10 parallel requests
⏱️  Timeout: 600s per request

📋 Requirements (Slot tersedia):
Area_Fungsi
Keamanan Informasi dan Siber        5270
Layanan Teknologi Informasi         3480
Sains Data-Kecerdasan Artifisial    2008
Tata Kelola Teknologi Informasi     3527
Teknologi dan Infrastruktur         3272
Name: sisa_slot, dtype: int64


🔄 Processing Part 1/9: Baris 1 - 500


Part 1: 100%|██████████| 500/500 [01:51<00:00,  4.47it/s]

  → Success: 500/500 rows | Failed: 0 rows


✅ Part 1 saved: /Users/irz/projects/dtp-data-pipeline/Koreksi_Parts_20251209_102415/part_001_rows_1-500.xlsx
   Progress: 500/4440 baris (11.3%)
   📊 Slot tersisa total: 17057

🔄 Processing Part 2/9: Baris 501 - 1000


Part 2: 100%|██████████| 500/500 [01:45<00:00,  4.76it/s]

...

Part 9:   0%|          | 0/440 [00:00<?, ?it/s]

[SKIP] Row 4033: Tidak ada slot tersedia untuk Area Fungsi 'Sains Data-Kecerdasan Artifisial'
[SKIP] Row 4062: Tidak ada slot tersedia untuk Area Fungsi 'Sains Data-Kecerdasan Artifisial'
[SKIP] Row 4103: Tidak ada slot tersedia untuk Area Fungsi 'Sains Data-Kecerdasan Artifisial'
[SKIP] Row 4132: Tidak ada slot tersedia untuk Area Fungsi 'Sains Data-Kecerdasan Artifisial'
[SKIP] Row 4384: Tidak ada slot tersedia untuk Area Fungsi 'Sains Data-Kecerdasan Artifisial'
[SKIP] Row 4019: Tidak ada slot tersedia untuk Area Fungsi 'Sains Data-Kecerdasan Artifisial'
[SKIP] Row 4065: Tidak ada slot tersedia untuk Area Fungsi 'Sains Data-Kecerdasan Artifisial'
[SKIP] Row 4116: Tidak ada slot tersedia untuk Area Fungsi 'Sains Data-Kecerdasan Artifisial'
[SKIP] Row 4136: Tidak ada slot tersedia untuk Area Fungsi 'Sains Data-Kecerdasan Artifisial'
[SKIP] Row 4156: Tidak ada slot tersedia untuk Area Fungsi 'Sains Data-Kecerdasan Artifisial'
[SKIP] Row 4378: Tidak ada slot tersedia untuk Area Fungsi '

Part 9: 100%|██████████| 440/440 [00:01<00:00, 240.78it/s]

  → Success: 0/440 rows | Failed: 440 rows


✅ Part 9 saved: /Users/irz/projects/dtp-data-pipeline/Koreksi_Parts_20251209_102415/part_009_rows_4001-4440.xlsx
   Progress: 4440/4440 baris (100.0%)
   📊 Slot tersisa total: 15549

🎉 Semua part selesai diproses!
📂 Part files tersimpan di: /Users/irz/projects/dtp-data-pipeline/Koreksi_Parts_20251209_102415
📋 Total part files: 9

🔗 Menggabungkan semua part files...
   Loading Part 1/9: part_001_rows_1-500.xlsx
   Loading Part 2/9: part_002_rows_501-1000.xlsx
   Loading Part 3/9: part_003_rows_1001-1500.xlsx
   Loading Part 4/9: part_004_rows_1501-2000.xlsx
   Loading Part 5/9: part_005_rows_2001-2500.xlsx
   Loading Part 6/9: part_006_rows_2501-3000.xlsx
   Loading Part 7/9: part_007_rows_3001-3500.xlsx
   Loading Part 8/9: part_008_rows_3501-4000.xlsx
   Loading Part 9/9: part_009_rows_4001-4440.xlsx

✓ Berhasil load 9 part files
✓ Total baris setelah penggabungan: 4440

✅ SELESAI!
📂 Part files: /Users/irz/projects/dtp-data-pipeline/Koreksi_Parts_20251209_102415
📄 Final file: /Users/i

In [58]:
# df = pd.read_excel(INPUT_FILE)
# df_not_processed = df.iloc[1000:].copy()

# # Tampilkan info
# print(f"Total baris yang dieksport: {len(df_not_processed)}")
# print(f"Dari baris 1001 (index 1000) hingga baris {len(df)} (index {len(df)-1})")

# # Eksport ke file Excel
# output_file = f"{DRIVE_DATASET_DIR}/Data_Diploy_Not_Corrected_7k.xlsx"
# df_not_processed.to_excel(output_file, index=False)
# print(f"✅ File tersimpan: {output_file}")

# df_not_processed.info()